# Santa Ada Bank

In [1]:
import csv

In [22]:
registros = []

In [23]:
def criar_registro(tipo: str, valor: float):
    
    registro = []
    TAXA = 0.02
    tempo = 10 # TODO: tempo depois tem que ser calculado e não fixo

    tipo_upper = tipo.upper()
    registro.append('31/12/9999')
    registro.append(tipo_upper)

    if tipo_upper == 'RECEITA':
        registro.append(valor)
        registro.append('')
    elif tipo_upper == 'DESPESAS':
        registro.append(-valor)
        registro.append('')
    elif tipo_upper == 'INVESTIMENTO':
        registro.append(valor)
        
        montante = valor * (1 + TAXA) ** tempo
        registro.append(montante)
    

    registros.append(registro)

In [24]:
criar_registro('receita', 500)
criar_registro('despesas', 100)
criar_registro('investimento', 10000)
print(registros)

[['31/12/9999', 'RECEITA', 500, ''], ['31/12/9999', 'DESPESAS', -100, ''], ['31/12/9999', 'INVESTIMENTO', 10000, 12189.944199947573]]


In [25]:
with open('registros.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';',
                            quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Data', 'Tipo', 'Valor', 'Montante'])
    for row in registros:
        spamwriter.writerow(row)

In [ ]:
def ler_registro():
    pass

def atualizar_registro():
    pass

def deletar_registro():
    pass

def atualizar_rendimento():
    pass

def exportar_relatorio():
    pass